# Chapter 9 - Implementation Tips

-------------------------------

When students start out with programming, they still have to discover certain standardized ways of implementation. If you stick to such standardized ways, you have a much easier time designing programs as you no longer need to think about certain details which are obvious to experienced programmers. In this chapter I will discuss some of these ways. The chapter is aimed specifically at Python programming, as other programming languages may have idiosynchracies which may make some of the ways discussed here less applicable.

## Choosing Data Structures

Data structures are constructs which allow you to store values . When you learn Python programming, initially there are three data structures which you focus on: single-value variables (integers, strings, and floating-points), lists, and dictionaries. You may occasionally encounter two more: tuples and sets. For novice programmers it is often unclear when one should use which data structure. There are some straightforwards rules-of-thumb which one can use to make decisions in this respect.

Obviously, when you need to store a single value, you use a single-value variable. That much is clear. But when do you use a list, dictionary, tuple, or set? Tuples and sets are used in exceptional cases, so let's first look at lists and dictionaries.

A __list__ is a data structure with an *ordering*. That means that if you have a series of values which you need to store, and you need to be able to say "this one is the first, this one is the second, this one is the third, etc." then a list is the obvious choice. Furthermore, since a list has an ordering, it can be *sorted*. None of the other standard data structures can be sorted. Thus, if you need sorting, you need a list.

A __dictionary__ is a data structure which has no ordering. It is used to store values which belong to a "key". Thus you use a dictionary when you need to store a series of values, which you can look up based on some key. Often, you think of dictionaries as data structures which store "pairs," namely a value tied to a key.

To give an example to distinguish the two, suppose that you have to read all the words in a file, and do something with them. Which data structure do you use? The answer is: that depends on *what* you need to do with them. If, for instance, you need to present all the words in the file in alphabetical order, then you will have to sort them at some point and thus a list is the obvious way of storing them. However, if you have to count how often each word occurs in the file, you have to store a word with a counter, i.e., you have to store key-value pairs, so a dictionary is the obvious way of storing them.

Here is some code which does both these things (if you want to practice with coding, you may wish to develop this code yourself before you look at how I do it; I have based my code on the code developed in the chapter on problem decomposition):


In [ ]:
def stringclean( inputstring ):
    outputstring = ""
    for letter in inputstring.lower():
        if letter >= 'a' and letter <= 'z':
            outputstring += letter
        else:
            outputstring += " "
    return outputstring

def get_wordlist( filename ):
    with open( filename ) as fp:
        contents = fp.read()
    wordlist = stringclean( contents ).split()
    wordlist.sort()
    return wordlist

def get_wordcounts( filename ):
    with open( filename ) as fp:
        contents = fp.read()
    wordcounts = {}
    wordlist = stringclean( contents ).split()
    for word in wordlist:
        wordcounts[word] = wordcounts.get( word, 0 )+1
    return wordcounts

print( "Word list:" )
wordlist = get_wordlist( "pc_woodchuck.txt" )
print( ", ".join( wordlist ) )
print()
print( "Word counts:" )
worddict = get_wordcounts( "pc_woodchuck.txt" )
for word in worddict:
    print( f"{word}: {worddict[word]}" )

If this difference is clear, then you may consider when to use a set or a tuple.

A __set__ is unordered, and each item in the set can occur only once. You may have noticed that in the code above, the list of words had words which occurred multiple times. Perhaps you do not want that. You may then consider using a set to store the words, because it will guarantee automatically that each word would be in the set just once. However, a set would *not* be a suitable approach for this, because I said that the words should be presented in alphabetical order. Since a set cannot be ordered, you cannot sort it. So when would you use a set?

The main reason to use a set is when you have to perform set operations. You can create a union of two sets, or create the intersection of two sets. Other set operations are possible as well. Thus if, for instance, I have two files and I am asked to list all the words that occur in both files, a good solution would be to put all the words of one file in one set, and all the words of the other file in another set, and then create the intersection of those two sets. Obviously, it does not happen often that you would need to perform such operations. Therefore the application of sets is rare.

So how would I solve the issue with listing all the words of a file in alphabetical order, but only list each word once? There are numerous solutions for this problem, of which I list two. The first is that before I store a word in the list, I check if it is already in the list, and only store it if it is not there yet. This is an acceptable solution, though relatively time-intensive, as checking whether something exists in a list is an expensive operation. The second solution is to simply build the list, then convert the list to a set, then convert the set back to the list, and then sort the list. The conversion to the set I do to get rid of doubles, and the conversion back to the list I do because I need to sort the list. Both these solutions are in the following code:

In [ ]:
def stringclean( inputstring ):
    outputstring = ""
    for letter in inputstring.lower():
        if letter >= 'a' and letter <= 'z':
            outputstring += letter
        else:
            outputstring += " "
    return outputstring

def get_wordlist_with_unique_test( filename ):
    with open( filename ) as fp:
        contents = fp.read()
    words = stringclean( contents ).split()
    wordlist = []
    for word in words:
        if word not in wordlist:
            wordlist.append( word )
    wordlist.sort()
    return wordlist

def get_wordlist_via_set( filename ):
    with open( filename ) as fp:
        contents = fp.read()
    wordlist = stringclean( contents ).split()
    wordset = set( wordlist )
    wordlist = list( wordset )
    wordlist.sort()
    return wordlist

wordlist = get_wordlist_with_unique_test( "pc_woodchuck.txt" )
print( ", ".join( wordlist ) )
wordlist = get_wordlist_via_set( "pc_woodchuck.txt" )
print( ", ".join( wordlist ) )


You may think of other solutions to the problem yourself. For instance, you can think of how you would use a dictionary with word counts to create an alphabetically ordered list of all the unique words.

Finally, a few words on tuples. 

A __tuple__ is an *ordered* data structure which is *immutable*. You may consider that a tuple is a list, but you cannot make changes to that list, e.g., you cannot sort a tuple. In general, almost any application for a tuple can just as well be done with a list, and a list can do much more than a tuple. The main exception is when you must use an immutable data structure somewhere. A good example is when you want to build a dictionary where each key consists of multiple values. Since the keys for a dictionary must be immutable, you cannot use lists for such keys; you must use tuples. However, this kind of application is pretty advanced, so you do not need to spend much thought on it at this time.

## Building Loops over Sequences

Functionally, there two kinds of loops: a loop that moves through a sequence of items (such as the items in a list), and a loop which continues for an indeterminate amount of time, until a certain conditions occurs.

The first kind of loop is usually (but not necessarily) implemented by a `for`-loop. The other kind of loop is discussed in the next section.

If the sequence of items you have to process are in a sequential data structure (list, dictionary, set, or even tuple), you can just write `for <item> in <sequence>`, and the code in the body of the loop will be run for each item in the sequence. Occasionally, you may need the index of the item in the sequence, and in that case you usually write the `for`-loop as processing the indices with a `range()`. 

For example:

In [ ]:
sequence = ['apple','pear','banana','grape','orange']

for item in sequence:
    print( item )

print()    
    
for i in range( len( sequence ) ):
    print( sequence[i] )

When you run this code, you see that both loops produce the same output. The first loop, however, does *not* have access to the indices of the items in the list.

For the second loop, please note the following: the variable `i` takes on the values that are produced by `range()`. `range()` gets as argument the length of the sequence. The values that it produces are integers 0, 1, 2, ... up to but *not* including the length of the sequence. In this case, the length of the sequence is 5 (it has five items), so the five values that `range()` produces here are 0, 1, 2, 3, and 4. 

This neatly lines up with how indices work in Python, and actually in almost all programming languages: they start at zero and go up to (but not including) the length of the sequence for which you use the indices. This becomes even more clear when you try to do the same thing as the second loop does, with a `while` loop.

In [ ]:
sequence = ['apple','pear','banana','grape','orange']

for i in range( len( sequence ) ):
    print( sequence[i] )
    
print()

i = 0
while i < len( sequence ):
    print( sequence[i] )
    i += 1

You can see that the `while` loop does exactly the same as the `for` loop, namely access the items of `sequence` by their index. Note that for the `while` loop, we start the index `i` at 0, and increase it until `i` gets equal to or becomes greater than the length of `sequence`.

This is the typical way of writing a loop which goes through items of a sequence by index. Start at index 0 and write a comparison which checks if this index is still smaller than the length of the sequence. *Always* write loops that process indices in this way. If you stick to this standard, you no longer have to spend any thought on whether you are counting one item too few or too many.

## Building Loops with an Indeterminate End

If you have to end a loop when "some condition occurs," but you cannot tell up front how many times the loop code should be run, you must use a `while` loop. There are two main ways to write a `while` loop. The first you use if the condition is a fairly straightforward test. 

For instance, suppose that I am searching for a number of 4 digits for which the sum of the digits is the same as the product of the digits (this was an exercise in a previous chapter). I have written two functions, `sum_of_digits()` and `product_of_digits()` that calculate the sum and product of the digits of a number, respectively. I can use them to write a `while` loop where the condition of the loop basically says: do this loop as long as you have not found a solution to the problem. I do not know exactly how many times I have to go through the loop, but once I found the solution, the loop ends (in fact, the body of the loop is executed 124 times).

In [ ]:
def sum_of_digits( num ):
    strnum = str( num )
    total = 0
    for c in strnum:
        total += int( c )
    return total

def product_of_digits( num ):
    strnum = str( num )
    total = 1
    for c in strnum:
        total *= int( c )
    return total

num = 1000
while sum_of_digits( num ) != product_of_digits( num ):
    num += 1
    
print( num )

Now, there is a bit of a problem with this code. The problem is that, before I solved the problem, I did not know if there actually was a 4-digit number for which the sum of the digits equals the product of the digits. If such number does not exist, the loop is endless!

One way of solving this is using a logical operator. If extend the condition of the loop with an extra clause, namely that the number should not be higher than 9999. So I do the loop while I have not found a solution *and* the number is still smaller than 10000. This I can solve with a logical `and`.

In [ ]:
def sum_of_digits( num ):
    strnum = str( num )
    total = 0
    for c in strnum:
        total += int( c )
    return total

def product_of_digits( num ):
    strnum = str( num )
    total = 1
    for c in strnum:
        total *= int( c )
    return total

num = 1000
while sum_of_digits( num ) != product_of_digits( num ) and num < 10000:
    num += 1
    
print( num )

While writing a loop with one or more logical operators in the condition works, such conditions tend to become unreadable quickly. That is why I often write them differently, namely by using `while True`, which means that the loop is endless, but then write conditions as the first statements in the loop which `break` out of the loop if certain conditions occur. This also tends to be easier to read as you can make positive rather than negative conditions, i.e., the loop ends when a solution is found, rather than the loop continues as long as the solution is not found.

In [ ]:
def sum_of_digits( num ):
    strnum = str( num )
    total = 0
    for c in strnum:
        total += int( c )
    return total

def product_of_digits( num ):
    strnum = str( num )
    total = 1
    for c in strnum:
        total *= int( c )
    return total

num = 1000
while True:
    if num > 9999:
        break
    if sum_of_digits( num ) == product_of_digits( num ):
        break
    num += 1
    
print( num )

The `while True` construction tends to be confusing to students, but it is not more than what I describe above: the conditions on which the loops ends are written at the top of the loop, with `break` statements to end the loop. It is also a good solution if you have to execute a function or do a calculation before you can test the condition. For instance, if you read lines from a file, and you should end the loop when there is nothing to read anymore, you first have to do a `readline()` before you can draw the conclusion that you are done.

In [ ]:
fp = open( "pc_woodchuck.txt" )
while True:
    buffer = fp.readline()
    if buffer == "": # There is nothing to read anymore
        break
    print( buffer, end="" )

## Initializing and Deinitializing

Often when you have to compute something, you need to do some initializing, like setting variables to an initial value, and some deinitializing, like using the computed values of the variables you initialized. I often see students having problems finding the right spot in the code where to initialize a variable, and where to use it.

If you know "the right spot" for one of them, you usually know the right spot for the other, as they tend be on the same indentation level. I.e., if I do some sort of calculation which produces a value in a variable, then I read out that variable right after the calculation, and the initialization of the variable almost always has to be at the same indentation level right before the calculation.

Let's give an example. Suppose that with a simulation I want to estimate the percentage of rolls of five six-sided dice that have a total value of 15 or more. This means I have to code a nested loop. Using bottom-up development, I first program the inner loop, where I roll five dice.

In [ ]:
from random import randint

total = 0 # initialization
for i in range( 5 ):
    total += randint( 1, 6 )
print( total ) # deinitialization

When I want to calculate a total value for five dice which are rolled, I need to roll a single die five times, and add up the values. For that I need a variable in which to store the total value, which I call `total`. That variable is initialized at 0, and can be read out after the loop is done. In the code you can see that setting the value of `total` to zero and having the value of total be the total of five dice, is at the same indentation level.

Now, I said that I wanted to estimate what percentage of rolls end up with a total of 15 or higher. Let's say that I simulate rolling the dice 1000 times. That means that I have to build a loop around my previous code, which executes that code 1000 times. I need to keep track of how many of those rolls are 15 or higher. So I have to keep track of that using a second variable, which I need to initialize to 0 before the simulation, and which will have the desired value after the simulation. So the code becomes:

In [ ]:
from random import randint

TRIALS = 1000

over15 = 0 # initialization 1
for j in range( TRIALS ):
    total = 0 # initialization 2
    for i in range( 5 ):
        total += randint( 1, 6 )
    if total >= 15: # deinitialization 2
        over15 += 1
print( over15/TRIALS ) # deinitialization 1

In this code I had to change the line where I printed the value of `total` to using this value to see if I need to increase `over15`, which keeps track of how many rolls were 15 or higher. This use of `total` is (of course) still at the same indentation level as the initialization of `total`. In the same vein, the initialization of `over15` is at the same level as the use of `over15`, in the last line of the code.

I often see students doing this wrong. Two typical errors that are made in the code that I developed above are initializing `total` too early and initializing it too late. If you initialize `total` too early, i.e., at the same spot where you initialize `over15`, `total` will never be reset to zero in the code and thus will be increased after every roll, which means that you end up with 0.999 or 1.000. If you initialize it too late, i.e., underneath the line `for i in range( 5 )`, you set total to zero before every individual die roll, and you end up with 0.000. 

It is a rule-of-thumb that applies almost always that, as I said, initialization and deinitialization are at the same indentation level.

---

## Exercises

### Exercise 9.1

You read a file with some contents, and you have to do something with those contents. In the following situations, which data structure would you use to store the contents of the file? Note: sometimes you may wish to use multiple data structures. Note 2: Sometimes there is a "best answer," but not always.

<ul>
<li>The file contains a list of items ordered by a client. Each item is listed with a quantity and a price. You must determine the total cost of the order.</li>
<li>The file contains a list of words that are legal in a particular language. You must use these words to spellcheck a text.</li>
<li>The file contains a list of matches in an eSports event. For each match is listed which two competitors engaged in the match, and who of them won the match. You must use the match information to produce a ranked list of all the competitors, from best-scoring to worst-scoring.</li>
<li>The file contains a list of bachelor courses with for each course the number of the course and the number of study points (ECTS). You have a series of files, each file for one student, which contains a list of the courses the student took (by course number), and their final grade for the course. You must determine which students are ready to receive their bachelor diploma (i.e., which students have a total of 180 study points of courses they passed).</li>
<li>The file contains a text which has, in several places, links to other texts which must be inserted in the given text. You have to produce the text which has all these links replaced by the texts that are to be inserted.</li>
</ul>

### Exercise 9.2

The file "wordsEn.txt" contains a list of words, one word per line. These are all legal words in the English language. Make a program that produces a list of all 5-letter words which contain an 'x'.

### Exercise 9.3

The file "wordsEn.txt" contains a list of words, one word per line. These are all legal words in the English language. Make a program that produces a list of all 5-letter words of which the letters are in alphabetical order. E.g., the word "abort" has all its letters in alphabetical order as 'a' < 'b' < 'o' < 'r' < 't'.

### Exercise 9.4

The file "wordsEn.txt" contains a list of words, one word per line. These are all legal words in the English language. Make a program that produces a list of all 5-letter words of which the letters are in alphabetical order and no letters are the same or right next to each other in the alphabet. E.g., the word "forty" meets the requirements as the letters are in alphabetical order and there is at least one other letter in the alphabet between any two of the letters.

### Exercise 9.5

The file "wordsEn.txt" contains a list of words, one word per line. These are all legal words in the English language. Make a program that produces a list of all 5-letter words of which the letters are in alphabetical order and at least two pairs of letters are right next to each other in the alphabet. E.g., the word "ghost" meets the requirements as the letters are in alphabetical order and the 'g' is next to the 'h', and the 's' is next to the 't'.

### Exercise 9.6*

The file "wordsEn.txt" contains a list of words, one word per line. These are all legal words in the English language. Make a program that produces a list of all 5-letter words which also have an anagram that is a legal 5-letter word. E.g., "earth" and "heart" would both be on the list, as they are anagrams of each other. An extra nice solution shows the anagrams next to each other, so "crate," "cater," "react," "caret," and "trace" (and perhaps also "recta" and "carte," if they are in the dictionary) are all on one line.

### Exercise 9.7*

The file "wordsEn.txt" contains a list of words, one word per line. These are all legal words in the English language. Make a program that produces a list of all words (regardless their number of letters) of which the letters are in alphabetical order. However, words that are part of other words on the list should be excluded. E.g., the word "most" should not be on the list as it is part of the word "almost". There are many ways to approach this exercise. Once you have found one, see if you can make it more efficient.



---

End of Chapter 9. Version 1.0.